# Julia and the Power of Multiple Dispatch

Julia is built around the concept of *multiple dispatch*. Multiple dispatch means that when you call a function 

```juila
f(a, b, c)
```

Julia chooses *which implementation* of the function `f` to call based on the types of `a`, `b`, and `c`. Compare this with Python, in which

```python
# python
f(a, b, c)
```

always calls the same function `f` for any `a`, `b`, and `c`, or

```python
# python
a.f(b, c)
```

which chooses the version of `f` based on the class of `a`, but not `b` or `c`. 

This allows us to do some really cool things in Julia that would be difficult in other languages. As an example, let's show off building a very primitive symbolic math tool. 

## Building a Symbolic Math Library

The first thing we need to do is define our data type. We'll call it an Expression and it will store a mathematical expression as a string. This is not a particularly good way to write a symbolic math tool, but it's useful as a demonstration:

In [1]:
type Expression
    data::String
end

Now we can construct variables: 

In [2]:
x = Expression("x")
y = Expression("y")

Expression("y")

Let's implement addition for our symbolic type. To do that, we overload the `+` function:

In [3]:
import Base: +

+(e1::Expression, e2::Expression) = Expression("$(e1.data) + $(e2.data)")

+ (generic function with 164 methods)

Now we can add two expressions:

In [4]:
x + y

Expression("x + y")

But what if we want to add other number types to our Expressions? This is where multiple dispatch comes in. We can define `+` for any combination of types that we want:

In [5]:
# Define adding an expression to a number:
+(e::Expression, n::Number) = Expression("$(e.data) + $(n)")

# Define adding a number to an expression:
+(n::Number, e::Expression) = Expression("$(n) + $(e.data)")

+ (generic function with 166 methods)

Now we can do:

In [6]:
x + 1

Expression("x + 1")

Aside: what is the `Number` type? `Number` is an *abstract* type representing any kind of scalar number. There are lots of subtypes of `Number` defined in Julia, and you can add your own, too:

In [7]:
# From https://en.wikibooks.org/wiki/Introducing_Julia/Types
function showtypetree(T, level=0)
    println("\t" ^ level, T)
    for t in subtypes(T)
        if t != Any
            showtypetree(t, level+1)
        end
   end
end

showtypetree(Number)

Number
	Complex{T<:Real}
	Real
		AbstractFloat
			BigFloat
			Float16
			Float32
			Float64
		Integer
			BigInt
			Bool
			Signed
				Int128
				Int16
				Int32
				Int64
				Int8
			Unsigned
				UInt128
				UInt16
				UInt32
				UInt64
				UInt8
		Irrational{sym}
		Rational{T<:Integer}


By defining `+` for our `Expression` and the abstract `Number` type, we have defined addition for all of the different kinds of numbers that Julia knows about:

In [8]:
# Integer
x + 1 

Expression("x + 1")

In [9]:
# Floating point
2.5 + x

Expression("2.5 + x")

In [10]:
# Rational
1//2 + x

Expression("1//2 + x")

In [11]:
# Complex
x + (1 + 3im)

Expression("x + 1 + 3im")

## Why Julia is different

We can do this kind of operator overloading in many languages. But the cool part about Julia is that there's nothing special about `+` or other operators. We can overload any function we want for any combination of types. Let's implement a few more functions for our Expression type:

In [12]:
import Base: sin, cos, *

sin(e::Expression) = Expression("sin($(e.data))")
cos(e::Expression) = Expression("cos($(e.data))")
*(e1::Expression, e2::Expression) = Expression("($(e1.data)) * ($(e2.data))")

* (generic function with 150 methods)

In [13]:
sin(x + y)

Expression("sin(x + y)")

In [14]:
x * y

Expression("(x) * (y)")

This is the basis for much of Julia's power and flexibility. We can define new interesting types, give them behaviors, and then use those behaviors to do more complex things. As a very basic demonstration, now that we've defined multiplication for our Expression type, we get sums and exponentiation for free:

In [15]:
(x + 1)^2

Expression("(x + 1) * (x + 1)")

In [16]:
sum([x, x^2, x^3])

Expression("x + (x) * (x) + (x) * ((x) * (x))")